Raspagem de dados do Diário Oficial da União buscando atos cuja autoria seja o órgão desejado

In [ ]:
#Importando Bibliotecas
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import math
import time

In [ ]:
#Função para tratamento de strings
def trata_html(input):
    return " ".join(input.split()).replace('> <', '><')

Variáveis a serem alteradas pelo usuário

In [ ]:
tipos = ['decreto', 'portaria', 'contrato', 'edital']

orgaos = ['Ministério da Saúde', 'Ministério da Defesa', 'Gabinete de Segurança Institucional', 
          'Ministério de Minas e Energia', 'Ministério da Infraestrutura', 
          'Ministério da Ciência, Tecnologia e Inovações', 'Casa Civil', 'Poder Executivo',
          'Vice-Presidência', 'Presidência']
url = "http://www.in.gov.br/consulta"
data_inicio = "2020-07-01"
data_fim = "2020-07-07"

In [ ]:
def primeira_maiuscula(lista):
    listab = []
    for palavra in lista:
        listab.append(palavra.lower().capitalize())
    return listab

In [ ]:
def gera_busca(r):
    
    html = trata_html(r.text)
    soup = BeautifulSoup(html, 'lxml')
    
    return soup

In [ ]:
#obter o total de paginas

def total_de_pgs(soup):

    tamanho = []
    for item in soup.find('div', class_='pagination-results').get_text().split():
        if(item.replace('.', '').isnumeric()):
            tamanho.append(item.replace('.', ''))
    pages = math.ceil(int(tamanho[-1]) / int(tamanho[-2]))
    return pages  

In [ ]:
#criando lista de links da busca

def busca_links(total_paginas, url):
    lista_urls = []

    for page in range(1, total_paginas):
        url_pg = url + '&start={}'.format(page)
        html = trata_html(requests.get(url_pg).text)
        soup = BeautifulSoup(html, 'lxml')
        lista_h5s = soup.findAll('h5', class_='title-marker')
        for h5 in lista_h5s:
            item = h5.find('a').get('href')
            tipo_de_doc = item.split('/-/')[-1].split('-')[0]
            if tipo_de_doc in tipos:
                lista_urls.append(item)
            
    return lista_urls
    

In [ ]:
#raspagem de dados
def raspa_dados(lista_de_links, ministerio):

    links = []
    autores = []
    chamadas = []
    datas = []
    ementas = []
    for i in lista_de_links:
        time.sleep(0.01)
        html = trata_html(requests.get(i).text)
        soup = BeautifulSoup(html, 'lxml')
        orgao = soup.findAll('span', class_="orgao-dou-data")[0].get_text()
        chamada = soup.findAll('p', class_="identifica")[0].get_text()
        data = soup.findAll('span', class_="publicado-dou-data")[0].get_text()
        ementa = soup.findAll('p', class_="ementa")
        if ministerio in orgao:
            autores.append(orgao)
            chamadas.append(chamada)
            datas.append(data)
            links.append(i)
            if len(ementa) >= 1:
              ementas.append(ementa[0].get_text())
            else:
              ementas.append(None)        
    return links, autores, chamadas, datas, ementas

In [ ]:
#função que gera planilha para cada integer da variavel 'orgaos'

def gera_df_orgao(orgao):
    print("Buscando dados de {}".format(orgao))
    params = {"q" : '"{}"'.format(orgao), "publishFrom" : data_inicio, "publishTo" : data_fim}
    r = requests.get(url, params = params)
    soup = gera_busca(r)
    total_paginas = total_de_pgs(soup)
    lista_urls = busca_links(total_paginas, r.url)
    links, ministerios, chamadas, datas, ementas = raspa_dados(lista_urls, orgao)
    data = {'orgao': ministerios, 'ato': chamadas, 'ementa': ementas, 'data': datas, 'link': links}
    planilha = pd.DataFrame(data)
    print("Dados obtidos de {} com sucesso".format(orgao))
    return planilha

In [ ]:
planilha = pd.concat([gera_df_orgao(i) for i in orgaos], ignore_index=True)

In [ ]:
planilha.to_excel('planilha_dou_{}.xlsx'.format(data_fim), index=False)